In [1]:
!pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/c1/0f/4dc0802131756a9fe4d46d2824352014b85a75baca386cb9e43057f39f15/mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for jax from https://files.pythonhosted.org/packages/fd/f2/9dbb75de3058acfd1600cf0839bcce7ea391148c9d2b4fa5f5666e66f09e/jax-0.4.30-py3-none-any.whl.metadata
  Obtaining dependency information for jaxlib from https://files.pythonhosted.org/packages/bb/1a/8f45ea28a5ca67e4d23ebd70fc78ea94be6fa20323f983c7607c32c6f9a5/jaxlib-0.4.30-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for sounddevice>=0.4.4 from https://files.pythonhosted.org/packages/d4/09/bfdd393f1bb1b90b4a6849b84972b7059c95e36818cc489922228d58cc63/sounddevice-0.4.7-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/50.8 MB 1.3 MB/s eta 0:00:39
   -----------------

### using hand_landmarks model for image

In [28]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# Loading image
image = cv2.imread(r"C:\Users\balas\Downloads\exp\hand_04.jpg")
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converting to RGB for MediaPipe
# Create hand landmark detection object
with mp_hands.Hands(
    min_detection_confidence=0.5,  #confidence threshold defines the confidence of the model to detect hand more conf will make easy detections but can overfit.
    min_tracking_confidence=0.5   #tracking confidence defines the conf with which the model tracks hand from frame to frame , more value can make easy tracking but can overfit.
) as hands:
  # Process the image
  results = hands.process(rgb_image)

  # Draw landmarks on the image
  if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

# Display the image with landmarks
cv2.imshow('Hand Landmarks', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### realtime analysis

In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# Loading video from cam
cap= cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,486)
pos=(30,40)
while True:
    ret,image = cap.read()
    image=cv2.flip(image,1)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converting to RGB for MediaPipe
# Create hand landmark detection object
    with mp_hands.Hands(
       min_detection_confidence=0.6,  #confidence threshold defines the confidence of the model to detect hand more conf will make easy detections but can overfit.
       min_tracking_confidence=0.6   #tracking confidence defines the conf with which the model tracks hand from frame to frame , more value can make easy tracking but can overfit.
    ) as hands:
  # Process the image
        results = hands.process(rgb_image)

  # Draw landmarks on the image
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

# Display the image with landmarks
    cv2.imshow('Hand landmark Detection', image)
    cv2.waitKey(4) & 0xff
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

## storing the hand landmarks data into dictionary

In [1]:
import mediapipe as mp
import numpy as np
import cv2

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence= 0.5, min_tracking_confidence= 0.6)

# Define hand gestures dictionary
hand_gestures = {}

def add_gesture(gesture_name, image):
    """
    Add a hand gesture to the dictionary.

    Args:
    - gesture_name (str): Name of the hand gesture.
    - image: Image of the hand gesture.
    """
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process image using MediaPipe Hands
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        # Get the first hand's landmarks
        hand_landmarks = results.multi_hand_landmarks[0]

        # Convert landmark positions to list
        landmark_positions = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

        # Store gesture and landmarks in dictionary
        hand_gestures[gesture_name] = {
            'landmarks': landmark_positions,
            'image': image  # Optional: Store the image for visualization
        }
        print(f"Gesture '{gesture_name}' added successfully.")
    else:
        print(f"Failed to detect hand landmarks for gesture '{gesture_name}'.")

def track_hand_gesture(image):
    """
    Track hand gestures from the given image.

    Args:
    - image: Image to process.

    Returns:
    - gesture_name (str): Name of the detected hand gesture or None if no match found.
    """
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process image using MediaPipe Hands
    results = hands.process(image_rgb)
    if results.multi_hand_landmarks:
        # Get the first hand's landmarks
        hand_landmarks = results.multi_hand_landmarks[0]
        # Convert landmark positions to list
        landmark_positions = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

        # Compare landmark positions with stored gestures
        for gesture_name, gesture_data in hand_gestures.items():
            # Calculate similarity between landmark positions (e.g., using mean squared error)
            similarity = calculate_similarity(landmark_positions, gesture_data['landmarks'])

            if similarity > 0.6:  # Adjust threshold value as needed
                return gesture_name

    return None

def calculate_similarity(landmark_positions1, landmark_positions2):
    """
    Calculate similarity between two lists of landmark positions.

    Args:
    - landmark_positions1 (list): First list of landmark positions.
    - landmark_positions2 (list): Second list of landmark positions.

    Returns:
    - similarity (float): Similarity value between 0 and 1.
    """
    #calculating mean squared error for landmarks
    mse = sum((a[0] - b[0])**2 + (a[1] - b[1])**2 + (a[2] - b[2])**2 for a, b in zip(landmark_positions1, landmark_positions2)) / len(landmark_positions1)
    return 1 - mse / (255**2 * 3)  # Normalize to [0, 1] range
image_fist = cv2.imread(r"C:\Users\balas\Downloads\imgs\fist_img.jpg")
add_gesture("fist", image_fist)
finger = cv2.imread(r"C:\Users\balas\Downloads\imgs\index_img2.jpg")
add_gesture("index", finger)
print(hand_gestures["fist"])
# cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     suc,image = cap.read()
#     if not suc:
#         break
#     img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    

Gesture 'fist' added successfully.
Gesture 'index' added successfully.
{'landmarks': [(0.4441797733306885, 0.5844573974609375, -1.1996103239653166e-06), (0.5972054600715637, 0.5235911011695862, -0.06126492843031883), (0.7319201231002808, 0.4189154803752899, -0.09767122566699982), (0.7057103514671326, 0.34351667761802673, -0.12831145524978638), (0.5852804183959961, 0.35544446110725403, -0.14296647906303406), (0.580881655216217, 0.27579283714294434, -0.05791746452450752), (0.6575481295585632, 0.2703073024749756, -0.12488192319869995), (0.6541796922683716, 0.36198604106903076, -0.16352048516273499), (0.6195948123931885, 0.3767053186893463, -0.19364206492900848), (0.45962557196617126, 0.2713480591773987, -0.06653392314910889), (0.5459734201431274, 0.29327327013015747, -0.13431352376937866), (0.5699087381362915, 0.38191378116607666, -0.15526582300662994), (0.5426502227783203, 0.38060635328292847, -0.16776295006275177), (0.36672863364219666, 0.29798629879951477, -0.08381178975105286), (0.454

## testing on realtime hand data

In [4]:
import cv2
cap = cv2.VideoCapture(0)
while cap.isOpened():
    suc,image = cap.read()
    if not suc:
        break
    img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    gesture_name = track_hand_gesture(img)
    if hand_gestures.get(gesture_name) is not None:
        print(gesture_name)
    cv2.imshow('gesture Detection', img)
    cv2.waitKey(4) & 0xff

fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist
fist


KeyboardInterrupt: 

In [1]:
import mediapipe as mp
import numpy as np
import cv2

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Define a dictionary to store gesture data
gesture_data = {}

# Define a function to store hand data for a specific gesture
def store_gesture_data(gesture_name, hand_landmarks):
    # Convert hand landmarks to a numpy array
    hand_data = np.array([(lmk.x, lmk.y, lmk.z) for lmk in hand_landmarks.landmark])
    
    # Store the hand data in the gesture_data dictionary
    if gesture_name not in gesture_data:
        gesture_data[gesture_name] = []
    gesture_data[gesture_name].append(hand_data)

# Example usage:
cap = cv2.VideoCapture(0)  # Use a webcam

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break
    
    # Convert the image to RGB and process it with MediaPipe Hands
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    # Check if hand landmarks are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Store hand data for a specific gesture (e.g., "thumbs_up")
            store_gesture_data("Thumbs Up", hand_landmarks)
        n=input("next?(y/n)")
        if(n=='y'):
            for hand_landmarks in results.multi_hand_landmarks:
            # Store hand data for a specific gesture (e.g., "thumbs_up")
                store_gesture_data("finger", hand_landmarks)
        else:
            break
    # Break the loop after storing data for a few frames
    # Break the loop after storing data for a few frames
    if "Thumbs Up" in gesture_data and len(gesture_data["Thumbs Up"]) >= 10:
        break
    cv2.imshow('gesture Detection', image)
    cv2.waitKey(4) & 0xff

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

# Print the stored gesture data
print(gesture_data)

next?(y/n)y
next?(y/n)n
{'Thumbs Up': [array([[ 9.48384285e-01,  7.30396926e-01, -7.36515062e-08],
       [ 8.86520326e-01,  7.20574021e-01, -2.11269427e-02],
       [ 8.32297027e-01,  6.62976742e-01, -2.97476836e-02],
       [ 8.06458056e-01,  5.95591247e-01, -3.55703235e-02],
       [ 7.91258097e-01,  5.40140629e-01, -3.87784168e-02],
       [ 8.93193305e-01,  5.78947961e-01, -1.56342033e-02],
       [ 8.73354435e-01,  5.08515596e-01, -2.71063335e-02],
       [ 8.49868774e-01,  4.95148242e-01, -3.54621969e-02],
       [ 8.38035285e-01,  5.00355005e-01, -3.95580605e-02],
       [ 9.24956381e-01,  5.65310478e-01, -1.08572766e-02],
       [ 9.02881682e-01,  4.94121373e-01, -1.49355652e-02],
       [ 8.78341079e-01,  4.81556028e-01, -1.69793125e-02],
       [ 8.68707359e-01,  4.85934019e-01, -1.91432796e-02],
       [ 9.51818585e-01,  5.61541975e-01, -8.79158918e-03],
       [ 9.34368849e-01,  4.90705699e-01, -1.40099730e-02],
       [ 9.07686770e-01,  4.77933258e-01, -9.70436353e-03],
 

In [3]:
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
X = np.array([data for gesture, data_list in gesture_data.items() for data in data_list])
y = np.array([gesture for gesture, data_list in gesture_data.items() for _ in data_list])
# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X.reshape(X.shape[0], -1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Train a KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Test the classifier
def recognize_gesture(hand_landmarks):
    hand_data = np.array([(lmk.x, lmk.y, lmk.z) for lmk in hand_landmarks.landmark])
    hand_data_reduced = pca.transform(hand_data.reshape(1, -1))
    prediction = knn.predict(hand_data_reduced)
    return prediction[0]

# Example usage:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            gesture = recognize_gesture(hand_landmarks)
            print(gesture)  # Output: thumbs_up
    cv2.imshow('gesture Detection', image)
    cv2.waitKey(4) & 0xff
cap.release()
cv2.destroyAllWindows()


Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up
Thumbs Up


KeyboardInterrupt: 